In [1]:
import pandas as pd
import psycopg2

In [2]:
connection_string = psycopg2.connect(host="mypostgresdb.cnm9emj4w74o.us-east-2.rds.amazonaws.com", port = 5432, database="project3", user="rootlh", password="password")

In [3]:
df = pd.read_sql_query('select * from post_info', con=connection_string)
df.head()

,type,posts
0,INTJ,'https://www.youtube.com/watch?v=8IEQpfA528M ...
1,INFP,"'It's one of my daily 'music doodles', one min..."
2,INFP,'NO. But I have dyscalculia. :X|||I've learned...
3,ISTP,"'288970 It's been a while, guys. Also I bleach..."
4,INFP,'29 and still have it. Never had a girlfriend....


In [4]:
pip install -U textblob

Requirement already up-to-date: textblob in c:\users\shiba\anaconda3\lib\site-packages (0.15.3)
Note: you may need to restart the kernel to use updated packages.


In [5]:
from textblob import TextBlob

In [6]:
df['words_per_comment'] = df['posts'].apply(lambda x: len(x.split())/50)
df['question_per_comment'] = df['posts'].apply(lambda x: x.count('?')/50)
df['excl_per_comment'] = df['posts'].apply(lambda x: x.count('!')/50)
df['upper_case'] = df['posts'].str.findall(r'[A-Z]').str.len()/50
# df['adj'] = df['posts'].str.findall(r'[A-Z]').str.len()/50
df[['polarity', 'subjectivity']] = df['posts'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
# df['emojis'] = df['posts'].str.findall(u'[\U0001f300-\U0001f650]|[\u2000-\u3000]').str.len()
df['ellipsis_per_comment'] = df['posts'].apply(lambda x: x.count('...')/50)

print(df.head(10))

   type                                              posts  words_per_comment  \
0  INTJ  'https://www.youtube.com/watch?v=8IEQpfA528M  ...              22.86   
1  INFP  'It's one of my daily 'music doodles', one min...              17.46   
2  INFP  'NO. But I have dyscalculia. :X|||I've learned...              13.30   
3  ISTP  '288970 It's been a while, guys. Also I bleach...              10.90   
4  INFP  '29 and still have it. Never had a girlfriend....              27.98   
5  INFJ  'It gets to be pretty obvious the lifestyle di...              20.98   
6  INTP  'I used to find this difficult until I realize...              23.80   
7  INTJ  '1. ENTJs - Without fail my favorite people.  ...              33.88   
8  ENFP  'Can I just say I'm so glad and thankful that ...              26.28   
9  ESFP  'No I'm not, you go home!|||Farnsworth: ENTP F...              21.80   

   question_per_comment  excl_per_comment  upper_case  polarity  subjectivity  \
0                  0.68    

In [7]:
df.describe()

,words_per_comment,question_per_comment,excl_per_comment,upper_case,polarity,subjectivity,ellipsis_per_comment
count,8675.000000,8675.000000,8675.000000,8675.000000,8675.000000,8675.000000,8675.000000
mean,24.524830,0.214619,0.169086,5.642501,0.125152,0.535031,0.662485
std,6.224211,0.139899,0.232219,1.893284,0.060422,0.045603,0.299458
min,0.080000,0.000000,0.000000,0.000000,-0.800000,0.000000,0.000000
25%,20.930000,0.120000,0.040000,4.480000,0.087922,0.505834,0.460000
50%,25.560000,0.200000,0.100000,5.560000,0.122788,0.533764,0.660000
75%,29.140000,0.280000,0.220000,6.660000,0.159827,0.562812,0.820000
max,37.620000,2.420000,4.380000,18.800000,0.558087,1.000000,3.920000


In [8]:
map1 = {"I": 0, "E": 1}
map2 = {"N": 0, "S": 1}
map3 = {"T": 0, "F": 1}
map4 = {"J": 0, "P": 1}

df['I-E'] = df['type'].astype(str).str[0]
df['I-E'] = df['I-E'].map(map1)

df['N-S'] = df['type'].astype(str).str[1]
df['N-S'] = df['N-S'].map(map2)

df['T-F'] = df['type'].astype(str).str[2]
df['T-F'] = df['T-F'].map(map3)

df['J-P'] = df['type'].astype(str).str[3]
df['J-P'] = df['J-P'].map(map4)

print(df.head(10))

   type                                              posts  words_per_comment  \
0  INTJ  'https://www.youtube.com/watch?v=8IEQpfA528M  ...              22.86   
1  INFP  'It's one of my daily 'music doodles', one min...              17.46   
2  INFP  'NO. But I have dyscalculia. :X|||I've learned...              13.30   
3  ISTP  '288970 It's been a while, guys. Also I bleach...              10.90   
4  INFP  '29 and still have it. Never had a girlfriend....              27.98   
5  INFJ  'It gets to be pretty obvious the lifestyle di...              20.98   
6  INTP  'I used to find this difficult until I realize...              23.80   
7  INTJ  '1. ENTJs - Without fail my favorite people.  ...              33.88   
8  ENFP  'Can I just say I'm so glad and thankful that ...              26.28   
9  ESFP  'No I'm not, you go home!|||Farnsworth: ENTP F...              21.80   

   question_per_comment  excl_per_comment  upper_case  polarity  subjectivity  \
0                  0.68    

In [44]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
X = df.drop(['type','posts','I-E','N-S','T-F','J-P'], axis=1).values

In [41]:
# Introvert vs. Extrovert
y = df['I-E'].values
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=45)

rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)


In [43]:
IE = rf.score(X_test, y_test)
IE

0.7662517289073306

In [45]:
y_predict = rf.predict(X_test)

In [47]:
accuracy_score(y_test,y_predict)

0.7662517289073306

In [48]:
#Intuitive vs. Sensing
y = df['N-S'].values
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=45)

rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)

In [49]:
NS = rf.score(X_test, y_test)
NS

0.8607653296449977

In [50]:
y_predict = rf.predict(X_test)

In [51]:
accuracy_score(y_test,y_predict)

0.8607653296449977

In [52]:
# Thinking vs. Feeling
y = df['T-F'].values
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=45)

rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)

In [53]:
TF = rf.score(X_test, y_test)
TF

0.6182572614107884

In [54]:
y_predict = rf.predict(X_test)

In [55]:
accuracy_score(y_test,y_predict)

0.6182572614107884

In [ ]:
# Judging vs. Perceiving
y = df['J-P'].values
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=45)

rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)

In [56]:
JP = rf.score(X_test, y_test)
JP

0.6182572614107884

In [57]:
y_predict = rf.predict(X_test)

In [58]:
accuracy_score(y_test,y_predict)

0.6182572614107884

In [63]:
df.to_csv('Data/data1.csv', index=False)